In [16]:
from typing import TypedDict
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.chat_models import ChatOllama
from pydantic import BaseModel, Field
import os 
import json
from langgraph.graph import StateGraph, END
from langgraph.types import Interrupt, Command

In [4]:
class LongTermMemory:
    def __init__(self, filepath = "memory1.json"):
        self.filepath = filepath
        if not os.path.exists(filepath):
            with open(filepath, "w", encoding = 'utf-8') as f:
                json.dump([], f)

    def load_lessons(self):
        with open(self.filepath, "r", encoding = 'utf-8') as f:
            return json.load(f)
        
    def save_lesson(self, lesson):
        lessons = self.load_lessons()
        if lesson not in lessons:
            lessons.append(lesson)
            with open(self.filepath, "w", encoding = 'utf-8') as f:
                json.dump(lessons, f, ensure_ascii = False, indent = 2)
            print(f"درس جدید در حافظه بلند مدت ذخیره شد: {lesson}")

In [5]:
memory_system = LongTermMemory()

In [6]:
class AgentState(TypedDict):
    question: str
    answer: str 
    feedback: str
    solved: bool 
    attempt_count: int

In [7]:
llm = ChatOllama(
        model = "llama3",
        temperature = 0.2
    )

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_14096\1501963825.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  llm = ChatOllama(


In [8]:
def generator_agent(state: AgentState):
    question = state["question"]
    attempt_count = state["attempt_count"]
    feedback = state.get("feedback", "")

    sys_msg = """
تو یک دستیار خبره در زمینه پاسخ دهی به سوالات کاربران در زمینه های مختلف هستی.
وظیفه تو این است که به سوالات کاربران در زمینه های مختلف، بهترین پاسخ را بدهی.
هدف تو ارائه یک پاسخ جامع و کامل به سوالات کاربران در زمینه های مختلف است.
فقط پاسخ را ارائه کن و از ارائه توضیحات خارج از پاسخ اجتناب کن.
"""

    hmn_msg = f"""
به سوال زیر به زبان فارسی پاسخ بده:
سوال: {question}
"""

    past_lessons = memory_system.load_lessons()
    if past_lessons:
        hmn_msg += "\nقوانین و درس های مهمی که از تجربیات قبلی یاد گرفتی که باید حتما آن ها را رعایت کنی:\n"
        for lesson in past_lessons:
            hmn_msg += f"- {lesson}\n"

    if feedback:
        hmn_msg += f"پاسخ قبلی که برای سوال کاربر تولید کردی با این بازخورد مواجه شد: {feedback}\nپاسخ را بهبود بده."
    
    message = [
        SystemMessage(content = sys_msg),
        HumanMessage(content = hmn_msg)
    ]

    response = llm.invoke(message).content

    print("=" * 100)
    print("\n")
    print(f"++++++++++ تلاش شماره {attempt_count + 1} ++++++++++\n")
    print(response)
    print("\n")

    return {
        "answer": response,
        "attempt_count": attempt_count + 1
    }

In [9]:
def validator_agent(state: AgentState):
    question = state["question"]
    answer = state["answer"]

    solved = False
    feedback = ""

    sys_msg = """
تو یک دستیار متفکر هستی که با استدلال دقیق و تکرار شونده، تفکر جریان سیال ذهن را تقلید می‌کنی، یعنی مانند انسان فکر میکنی. رویکرد تو بر کاوش، شک به خود و اصلاح مداوم قبل از رسیدن به پاسخ تاکید دارد.
وظیفه تو این است که پاسخ ارائه شده بر اساس سوال کاربر را از ابعاد مختلف، بررسی و تجزیه و تحلیل کنی و با خودت تفکر و استدلال کنی که اشکالات و نواقص پاسخ می تواند چه مواردی باشند و چگونه این پاسخ را بهبود می توان داد تا دقیق تر، جامع تر و کامل تر باشد و تمام نیازهای کاربر را بر اساس سوالی که پرسیده است، در بر گیرد. در نهایت، از 0 تا 100 یک امتیاز کلی به پاسخ ارائه شده، ارائه کن. 0 به معنای ضعیف ترین پاسخ و 100 به معنای عالی ترین پاسخ است.
هدف، ارائه یک راه حل دقیق و کامل برای بهبود پاسخ بر اساس سوال کاربر است به طوری که در نهایت امتیاز پاسخ به بالاتر از 80 برسد.
به زبان فارسی تمامی این فرآیندها را انجام بده.
"""

    hmn_msg = f"""
سوال کاربر: {question}

پاسخ ارائه شده: {answer}
"""
    
    message = [
        SystemMessage(content = sys_msg),
        HumanMessage(content = hmn_msg)
    ]

    response = llm.invoke(message).content

    print("=" * 100)
    print(f"++++++++++ ارزیابی پاسخ ارائه شده ++++++++++\n{response}")
    print("\n")

    prompt = f"""
بر اساس متن زیر، امتیاز دریافتی که متن زیر ارائه داده است را بررسی کن. 
اگر در متن زیر، امتیاز ارائه شده بیشتر از 80 بود، فقط عبارت "True" را چاپ کن و اگر امتیاز ارائه شده کمتر از 80 بود، فقط عبارت "False" را چاپ کن.

متن: {response}
"""
    
    response2 = llm.invoke(prompt).content 

    print("=" * 100)
    print(f"آیا امتیاز پاسخ ارائه شده بیشتر از 80 می باشد: {response2}")
    print("\n")

    try:
        if response2 == "False":
            raise ValueError(f"پاسخی که ارائه کردی دقت، جامع و کامل بودن کافی را ندارد. برای رسیدن به پاسخی دقیق تر، جامع تر و کامل تر دستورالعمل زیر را دنبال کن و در پاسخ های بعدی که ارائه می دهی، آن را در نظر بگیر تا پاسخی که ارائه می دهی، امتیازی بالاتر از 80 از نظر دقت، جامعیت و کامل بودن دریافت کند.\nدستورالعمل: {response}")
        print("پاسخ نهایی تولید شد")
        solved = True 
        final_answer = answer
        print("=" * 100)
        print(f"++++++++++ پاسخ نهایی ++++++++++")
        print(f"\n{final_answer}")

    except Exception as e:
        feedback = str(e)

    return {
        "solved": solved,
        "feedback": feedback
    }

In [10]:
def reflector_agent(state: AgentState):
    feedback = state["feedback"]

    prompt = (
        f"دستورالعمل برای بهبود پاسخ به سوال کاربر شامل این موارد است: {feedback}"
        "یک قانون کلی کوتاه از این دستورالعمل برای بهبود پاسخ از نظر دقت، جامعیت و کامل بودن استخراج کن."
        "تمامی این فرآیند را به زبان فارسی انجام بده."
    )

    lesson = llm.invoke([HumanMessage(content = prompt)]).content 

    print("=" * 100)
    print(f"++++++++++ قانون کلی برای بهبود پاسخ ++++++++++\n{lesson}")

    memory_system.save_lesson(lesson)

    return {}

In [ ]:
def router(state: AgentState):
    if state["solved"]:
        return END 
    if state["attempt_count"] > 3:
        return END 
    return "reflector_agent"

In [12]:
workflow = StateGraph(AgentState)

workflow.add_node("generator_agent", generator_agent)
workflow.add_node("validator_agent", validator_agent)
workflow.add_node("reflector_agent", reflector_agent)

workflow.set_entry_point("generator_agent")
workflow.add_edge("generator_agent", "validator_agent")
workflow.add_conditional_edges("validator_agent", router, {
    END: END,
    "reflector_agent": "reflector_agent"
})
workflow.add_edge("reflector_agent", "generator_agent")

app = workflow.compile()

In [13]:
def run_agent_test(question: str):
    initial_state = {
        "question": question,
        "answer": "",
        "feedback": "",
        "solved": False,
        "attempt_count": 0
    }

    final_state = app.invoke(initial_state)

    print("\n====== نتیجه نهایی ======")
    print(f"Solved: {final_state.get('solved')}")
    print(f"Attempts: {final_state.get('attempt_count')}")
    #print(f"Last feedback: {final_state.get('feedback')}")

    print("\n====== حافظه بلندمدت ======")
    lessons = memory_system.load_lessons()
    for i, lesson in enumerate(lessons, 1):
        print(f"{i}. {lesson}")

    return final_state

In [14]:
response = run_agent_test("یک مقاله 200 کلمه ای درمورد هوش مصنوعی به زبان فارسی برایم بنویس.")



++++++++++ تلاش شماره 1 ++++++++++

**هوش مصنوعی: آینده فناوری**

هوش مصنوعی (Artificial Intelligence) یک حوزه تحقیقاتی است که هدف آن ایجاد سیستم‌های هوشمند و خودکار است. این فناوری می‌تواند بر اساس الگوریتم‌ها و داده‌های جمع‌آوری شده، تصمیم‌گیری کند و اقدامات مختلف را انجام دهد.

هوش مصنوعی در چند دهه گذشته به سرعت پیشرفت کرده است. از یک سو، این فناوری در صنایع مختلف مانند پزشکی، بانکداری و بازرگانی استفاده می‌شود. از سوی دیگر، هوش مصنوعی در زندگی روزمره ما نیز حضور دارد. برای مثال، سیستم‌های صوتی هوشمند مثل آمازون الکسا و گугل هوم به کاربران اجازه می‌دهند که با صدای خود دستورات را اجرا کنند.

هوش مصنوعی همچنین می‌تواند بر اساس داده‌های جمع‌آوری شده، پیش‌بینی‌ها و تحلیل‌ها را انجام دهد. این فناوری در صنایع مختلف مانند بازرگانی، بانکداری و بیمه استفاده می‌شود. برای مثال، هوش مصنوعی می‌تواند پیش‌بینی فروش محصولات را انجام دهد و یا ریسک‌های مالی را ارزیابی کند.

با پیشرفت سریع هوش مصنوعی، آینده فناوری نیز تغییر خواهد کرد. هوش مصنوعی می‌تواند بر اساس داده‌های جمع‌آوری شده، تصمیم‌گیری کن

In [3]:
if os.path.exists("memory1.json"):
    os.remove("memory1.json")
    print("حافظه پاک شد. حالا می‌توانید دوباره از تسک ۱ شروع کنید.")

حافظه پاک شد. حالا می‌توانید دوباره از تسک ۱ شروع کنید.


In [39]:
response2 = run_agent_test("سلام، حال شما خوبه؟")



++++++++++ تلاش شماره 1 ++++++++++

خوشحال!


++++++++++ ارزیابی پاسخ ارائه شده ++++++++++
بررسی و تجزیه و تحلیل پاسخ:

پاسخ ارائه شده ("خوشحال!") به نظر می‌رسد که فقط یک پاسخ کوتاه و ساده است. این پاسخ در واقع هیچ اطلاعات یا تفکری درباره وضعیت کاربر ندارد. پاسخ باید بیشتر از آنکه فقط بگوید "خوشحال!" باشد.

اشکالات و نواقص پاسخ:

1. پاسخ خیلی ساده و کم‌اطلاعات است.
2. پاسخ هیچ اطلاعات یا تفکری درباره وضعیت کاربر ندارد.
3. پاسخ نمی‌تواند نیازهای کاربر را برآورده کند.

راه حل برای بهبود پاسخ:

1. سؤال کاربر را بهتر بفهمیم و در پاسخ خودمان به آن توجه کنیم.
2. پاسخ باید بیشتر از آنکه فقط بگوید "خوشحال!" باشد و اطلاعات بیشتری درباره وضعیت کاربر ارائه دهد.
3. پاسخ باید تفکری داشته باشد و نیازهای کاربر را برآورده کند.

امتیاز پاسخ: 20

هدف، ارائه یک راه حل دقیق و کامل برای بهبود پاسخ بر اساس سوال کاربر است به طوری که در نهایت امتیاز پاسخ به بالاتر از 80 برسد.


آیا امتیاز پاسخ ارائه شده بیشتر از 80 می باشد: False


++++++++++ قانون کلی برای بهبود پاسخ ++++++++++
بر اساس دستورالعمل ارائه شده

In [52]:
response3 = run_agent_test("چگونه می توانم از سئو درآمد عالی کسب کنم؟")



++++++++++ تلاش شماره 1 ++++++++++

To earn high income from SEO, consider the following strategies:

1. **High-traffic keywords**: Target competitive keywords with high search volume to attract a large audience.
2. **Niche expertise**: Focus on a specific niche and become an authority in that area to attract high-paying clients or advertisers.
3. **Content quality**: Create high-quality, engaging, and informative content that resonates with your target audience.
4. **Link building**: Build high-quality backlinks from authoritative sources to increase your website's credibility and search engine rankings.
5. **Conversion optimization**: Optimize your website for conversions by improving user experience, reducing bounce rates, and increasing form submissions.
6. **Affiliate marketing**: Promote affiliate products or services related to your niche and earn commissions for each sale made through your unique referral link.
7. **Consulting**: Offer SEO consulting services to businesses an